In [7]:
import torch.nn as nn
import torch
from torch import optim
import tqdm


In [14]:
import ast
from data_process import MLPDataset_For_League
import pandas as pd

In [63]:
class CNN(nn.Module):
    def __init__(self, bert):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels= 5, kernel_size = (2, 2), padding = 1)
        self.pool1 = nn.MaxPool2d(kernel_size = (1, 2))
        self.conv2 = nn.Conv2d(in_channels = 5, out_channels = 10, kernel_size = (5, 2), dilation = (2, 1))
        # self.pool2 = nn.MaxPool2d(kernel_size = (2, 1))
        self.relu = nn.ReLU()
        self.linear = nn.Linear(10 * 3 * 15, 2)

        self.bert = bert

        self.bert.eval()
        for param in self.bert.parameters():
            param.requires_grad = False


    def forward(self, x, segment_label):
        # get word embedding based on the champ Id
        embedded_x = self.bert.embedding(x, segment_label)
        
        # delete [SEP] tokens from input
        embedded_x = torch.cat((embedded_x[:, 1:6], embedded_x[:, 7:-1]), dim = 1)
        embedded_x = embedded_x.unsqueeze(1)

        # print()
        x = self.pool1(self.relu(self.conv1(embedded_x)))
        x = self.relu(self.conv2(x))
        x = x.view(x.size(0), -1)
        output = self.linear(x)
        return output
    

In [64]:
class CNN_Trainer:
    def __init__(
        self, 
        model, 
        train_dataloader, 
        test_dataloader=None, 
        lr= 1e-4,
        device='cuda'
        ):

        self.model = model
        self.train_data = train_dataloader
        self.test_data = test_dataloader
        self.device = device

        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(model.parameters(), lr = lr)

    def train(self, epoch):
        self.iteration(epoch, self.train_data)

    def test(self, epoch):
        self.iteration(epoch, self.test_data, train=False)


    def iteration(self, epoch, data_loader, train = True):
        avg_loss = 0.0
        total_correct = 0
        total_element = 0
        
        mode = "train" if train else "test"

        # progress bar
        data_iter = tqdm.tqdm(
            enumerate(data_loader),
            desc="EP_%s:%d" % (mode, epoch),
            total=len(data_loader),
            bar_format="{l_bar}{r_bar}"
        )

        for i, data in data_iter:

            # 0. batch_data will be sent into the device(GPU or cpu)
            data = {key: value.to(self.device) for key, value in data.items()}
            
            # 1. forward the input data to get output
            winner_output = self.model.forward(data["bert_input"], data["segment_label"])
            
            print(winner_output.shape)
            # 2-1. Crossentroyp loss of winner classification result
            loss = self.criterion(winner_output, (data["winner_label"]))

            # 3. backward and optimization only in train
            if train:
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()

            # next sentence prediction accuracy
            correct = winner_output.argmax(dim=-1).eq(data["winner_label"]).sum().item()
            avg_loss += loss.item()
            total_correct += correct
            total_element += data["winner_label"].nelement()

            post_fix = {
                "epoch": epoch,
                "iter": i,
                "avg_loss": avg_loss / (i + 1),
                "avg_acc": total_correct / total_element * 100,
                "loss": loss.item()
            }

            if i % 10 == 0:
                data_iter.write(str(post_fix))
        print(
            f"EP{epoch}, {mode}: \
            avg_loss={avg_loss / len(data_iter)}, \
            total_acc={total_correct * 100.0 / total_element}"
        ) 

In [65]:
df = pd.read_csv("../Data/match_data_2.csv")
df["teams"] = df["teams"].apply(lambda x : ast.literal_eval(x))


champ_idx = {}
with open("champ_idx.txt", 'r') as file:
    for line in file:
        pair = ast.literal_eval(line)
        champ_idx.update({pair[0]: pair[1]})



for i in range(len(df["teams"])):
    for j in range(len(df["teams"][i])):
        df["teams"][i][j] = champ_idx[df["teams"][i][j]]


train_datas = [[df["teams"][i][:5], df["teams"][i][5:], df["winner"][i]] for i in range(len(df))]

In [66]:
from torch.utils.data import DataLoader
from BERT import BERT,BERTLM

In [67]:
MAX_LEN = 13
vocab_size = 171

train_data = MLPDataset_For_League(
train_datas, seq_len=MAX_LEN)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_loader = DataLoader(
train_data, batch_size=16, shuffle=True, pin_memory=True)

bert_model = BERT(
vocab_size=vocab_size,
d_model= 32,
n_layers=8,
heads=8,
dropout=0.1,
device = device
)

bert_ = BERTLM(bert_model, vocab_size)
bert_.load_state_dict(torch.load("../Trained_Model/bert_model_final"))


wpm = CNN(bert_).to(device)
wpm_trainer = CNN_Trainer(wpm, train_loader, lr= 0.001, device='cuda')


prev_epochs = 0
epochs = 10
for epoch in range(prev_epochs, epochs):
    wpm_trainer.train(epoch)

torch.save(wpm.state_dict(), "../Trained_Model/cnn_model")

EP_train:0:   1%|| 14/1875 [00:00<00:26, 70.51it/s]

torch.Size([16, 2])
{'epoch': 0, 'iter': 0, 'avg_loss': 0.7408577799797058, 'avg_acc': 50.0, 'loss': 0.7408577799797058}
torch.Size([16, 2])
torch.Size([16, 2])
torch.Size([16, 2])
torch.Size([16, 2])
torch.Size([16, 2])
torch.Size([16, 2])
torch.Size([16, 2])
torch.Size([16, 2])
torch.Size([16, 2])
torch.Size([16, 2])
{'epoch': 0, 'iter': 10, 'avg_loss': 0.6916509053923867, 'avg_acc': 56.25, 'loss': 0.6062567830085754}
torch.Size([16, 2])
torch.Size([16, 2])
torch.Size([16, 2])


EP_train:0:   1%|| 22/1875 [00:00<00:25, 73.78it/s]

torch.Size([16, 2])
torch.Size([16, 2])
torch.Size([16, 2])
torch.Size([16, 2])
torch.Size([16, 2])
torch.Size([16, 2])
torch.Size([16, 2])
{'epoch': 0, 'iter': 20, 'avg_loss': 0.7079507055736723, 'avg_acc': 50.89285714285714, 'loss': 0.7040508389472961}
torch.Size([16, 2])
torch.Size([16, 2])
torch.Size([16, 2])
torch.Size([16, 2])
torch.Size([16, 2])
torch.Size([16, 2])
torch.Size([16, 2])


EP_train:0:   2%|| 38/1875 [00:00<00:24, 74.59it/s]

...

EP_train:1:  53%|| 985/1875 [00:15<00:18, 49.36it/s]

torch.Size([16, 2])
torch.Size([16, 2])
{'epoch': 1, 'iter': 980, 'avg_loss': 0.6846047549315792, 'avg_acc': 55.77854230377166, 'loss': 0.6785604357719421}
torch.Size([16, 2])
torch.Size([16, 2])
torch.Size([16, 2])
torch.Size([16, 2])
torch.Size([16, 2])
torch.Size([16, 2])
torch.Size([16, 2])
torch.Size([16, 2])
torch.Size([16, 2])
torch.Size([16, 2])


EP_train:1:  53%|| 996/1875 [00:15<00:13, 63.58it/s]


{'epoch': 1, 'iter': 990, 'avg_loss': 0.6846004826750692, 'avg_acc': 55.78960645812311, 'loss': 0.6978784203529358}
torch.Size([16, 2])
torch.Size([16, 2])
torch.Size([16, 2])
torch.Size([16, 2])
torch.Size([16, 2])


KeyboardInterrupt: 